In [1]:
import numpy as np
import sympy

In [2]:
perch_length = np.array(
    [8.4, 13.7, 15.0, 16.2, 17.4, 18.0, 18.7, 19.0, 19.6, 20.0, 
     21.0, 21.0, 21.0, 21.3, 22.0, 22.0, 22.0, 22.0, 22.0, 22.5, 
     22.5, 22.7, 23.0, 23.5, 24.0, 24.0, 24.6, 25.0, 25.6, 26.5, 
     27.3, 27.5, 27.5, 27.5, 28.0, 28.7, 30.0, 32.8, 34.5, 35.0, 
     36.5, 36.0, 37.0, 37.0, 39.0, 39.0, 39.0, 40.0, 40.0, 40.0, 
     40.0, 42.0, 43.0, 43.0, 43.5, 44.0]
     )
perch_weight = np.array(
    [5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 
     110.0, 115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 
     130.0, 150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 
     197.0, 218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 
     514.0, 556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 
     820.0, 850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 
     1000.0, 1000.0]
     )

In [3]:
class Rnn:
  """
  순환 신경망의 계산
  """
  # 입력 가중치
  input_w = []

  # 이전 출력에 대한 가중치
  before_w = []

  # 출력 가중치
  result_w = []

  # 편향 값 bais
  bias = []

  def cal_rnn(self, input, output, input_w = [], bias = []):
    """
    순환 신경망 은닉층까지의 계산
    args
      input : 입력 데이터
      output : 원하는 출력 형태
    return 
      순환 신경망의 계산 결과
    """
    # 이전 출력에 대한 입력, 첫 부분의 경우 이 값이 0이다.
    before = np.zeros((output.shape[0], output.shape[1]))

    # 이전 출력에 대한 가중치
    before_w = np.random.rand(output.shape[0], output.shape[0])

    self.before_w.append(before_w)

    # 가중치 w의 생성, 
    if not input_w:
      input_w = np.random.rand(output.shape[0], input.shape[0])
    
    if not bias:
      bias = np.random.rand()

    self.input_w.append(input_w)

    self.bias.append(bias)

    return (before_w @ before) + (input_w @ input) + bias

  def cal_result(self, input, output, w = []):
    """
    순환 신경망의 은닉층 출력에서 최종 결과값으로의 계산
    args
      input : 입력 데이터
      output : 원하는 출력 형태
    return 
      행렬 곱 연산 결과
    """
    # 가중치 생성
    if not w:
      result_w = np.random.rand(output.shape[0], output.shape[0])

    self.result_w.append(result_w)

    return result_w @ input

In [4]:
class Activation:
  """
  데이터의 활성화 함수 연산 결과를 얻기 위함
  추가로 역전파 계산에 필요한 활성화 함수의 미분 함수
  """
  # 활성화 함수
  fx = 0
  # 활성화 함수의 미분 함수
  d_fx = 0

  def sigmoid_fun(self, input):
    """
    시그모이드 함수
    자연 상수의 지수 함수를 사용한 0~1 사이의 값 출력
    Args:
      input : 입력값
    Return:
      0~1 사이의 값
    """
    x = sympy.Symbol('x')
    result = []

    self.fx = 1/(1+np.e**(-x))

    self.d_fx = sympy.diff(self.fx, x)

    for i in range(input.shape[0]):
      result.append(self.fx.subs([(x, input[i][0])]))

    return np.array(result).reshape(-1, 1)

  def d_sigmoid_fun(self, input):
    """
    시그모이드 함수의 미분값, 노드의 출력이 입력된다.
    arg
      input : 입력값
    return
      시그모이드 미분 함수의 계산 값
    """
    result = []

    x = sympy.Symbol('x')

    for i in range(input.shape[0]):
      result.append(self.d_fx.subs([(x,input[i][0])]))

    return np.array(result).reshape(-1, 1)

In [5]:
class Error:
  """
  오차 함수와 오차 함수의 미분 함수를 얻을 수 있다.  
  """
  # 오차 함수
  fx = 0
  
  # 오차 함수의 미분값
  d_fx = 0
  
  def RMSE(self, target, predict):
    """
    일반적인 제곱 오차 함수
    args:
      target : 목표 변수
      predict : 예측 변수
    return:
      제곱 오차 평균 값
    """
    result = (predict - target)**2
    self.result = np.mean(result)

    t = sympy.Symbol('t')
    y = sympy.Symbol('y')

    self.fx = 1/2*((y-t)**2)

    self.d_fx = sympy.diff(self.fx, y)

    return self.result

  def d_RMSE(self, target, predict):
    """
    RMSE 의 미분 함수
    args
      target : 목표 변수
      predict : 예측 변수
    return:
      미분 함수의 계산 값
    """
    result = []

    t = sympy.Symbol('t')
    y = sympy.Symbol('y')

    for i in range(target.shape[0]):
      result.append(self.d_fx.subs([(t, target[i][0]), (y, predict[i][0])]))
    
    return np.array(result).reshape(-1, 1)


In [6]:
input = perch_length[:3].reshape(-1,1)

In [7]:
target = perch_length[:4].reshape(-1,1)

In [8]:
output = np.zeros((4,1))

In [9]:
input.shape

(3, 1)

In [10]:
output.shape

(4, 1)

In [11]:
rnn = Rnn()

In [12]:
rnn_hidden_input = rnn.cal_rnn(input, output)
rnn_hidden_input

array([[28.23363806],
       [19.76318626],
       [18.51853987],
       [29.09123389]])

In [13]:
act = Activation()

In [14]:
rnn_hidden_output = act.sigmoid_fun(rnn_hidden_input)
rnn_hidden_output

array([[0.999999999999453],
       [0.999999997388096],
       [0.999999990932234],
       [0.999999999999768]], dtype=object)

In [15]:
rnn_result = rnn.cal_result(rnn_hidden_output, output)
rnn_result

array([[2.05354486281113],
       [2.27257113816046],
       [2.27612420071436],
       [2.41479886422985]], dtype=object)

델타 연산

예측 노드의 델타,

오차 함수의 미분값과 동일하다.(활성화 함수로 항등 함수를 사용했기 때문)

In [16]:
err = Error()

In [17]:
result = err.RMSE(target, rnn_result)
result

130.698102227510

In [22]:
result_delta = err.d_RMSE(target, rnn_result)
result_delta

array([[-6.34645513718887],
       [-11.4274288618395],
       [-12.7238757992856],
       [-13.7852011357702]], dtype=object)

예측 노드의 델타를 사용한 result_w 값 업데이트

In [19]:
rnn_hidden_output

array([[0.999999999999453],
       [0.999999997388096],
       [0.999999990932234],
       [0.999999999999768]], dtype=object)

In [23]:
d_result_w = result_delta * rnn_hidden_output
d_result_w

array([[-6.34645513718540],
       [-11.4274288319922],
       [-12.7238756839085],
       [-13.7852011357670]], dtype=object)